 # Notebook description

 This notebook is a counter-analysis of flight data and delays.

## Notepad configuration

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd 
import plotly.express as px
import os
from dotenv import  load_dotenv

In [2]:
load_dotenv("D:\Python\devcontainer\.env") #loading postgre password from .env file

True

Create url and engine

In [3]:
url = URL.create(
    "postgresql+psycopg2",
    username='postgres',
    password=os.getenv("POSTGRES_PASSWORD"),  
    host='localhost',
    database='postgres',
)
engine = create_engine(url)

Loading an SQL database using Pandas

In [198]:
def read_sql_table(table_name):
    df = pd.read_sql(sql=table_name, con=engine)
    return df

In [4]:
flight_df = pd.read_csv(r'flight_df_01.csv') #loading file flight_df_01.csv created at the end of the notebook 04_Data_Analysis_01.ipynb 

 ## Loading the `aircraft` table


In [201]:
aircraft_df = read_sql_table(table_name='aircraft')

Delete `number_of_seats`, `id` and duplicate columns from the `aircraft_df` frame.

In [202]:
aircraft_df = aircraft_df.drop(columns=['id', 'number_of_seats'])

In [203]:
aircraft_df = aircraft_df.drop_duplicates()

Check if there is more than one year of production for a given `tail_num`.

In [205]:
aircraft_df.duplicated('tail_num').any()

True

In [206]:
aircraft_df_duplicated = aircraft_df.loc[aircraft_df.duplicated(subset='tail_num')]

In [207]:
aircraft_df_duplicated.head()

,manufacture_year,tail_num
2086,2000,N783CA
5725,2014,N856GT
6746,2017,N877AS


 ## Modification `aircraft_df`.


For duplicated `tail_num`, `manufacture_year` has been set as the highest

In [209]:
aircraft_df['manufacture_year'] = aircraft_df.groupby('tail_num')['manufacture_year'].transform('max')

 ## Merge `aircraft_df` and `flight_df`.

Merged tables saved to `tmp_flight_df`.

In [212]:
tmp_flight_df = flight_df.merge(aircraft_df, how='left', on='tail_num')

Check for duplicates

In [213]:
tmp_flight_df.duplicated().any()

True

In [214]:
tmp_flight_df.shape

(1057408, 32)

In [215]:
tmp_flight_df = tmp_flight_df.drop_duplicates()

In [216]:
tmp_flight_df.shape

(1057391, 32)

Overwrite the `flight_df` table using `tmp_flight_df`.

In [217]:
flight_df = tmp_flight_df

 ## Delays vs. year of aircraft production
Determination of the relation of the year of production to the frequency of delays

In [218]:
delays_by_manufacture_year_df = flight_df.groupby('manufacture_year')['is_delayed'].mean().reset_index()

Chart Manufacture Year vs. Delay

In [219]:
fig = px.scatter(delays_by_manufacture_year_df, 
                    x = 'manufacture_year',
                    y = 'is_delayed',
                    title= 'Manufacture Year vs. Delay'
                   )

fig.update_xaxes(title_text='Manufacture Year')

fig.update_yaxes(title_text='Delayed Flights [%]')

fig.show()

Chart modification: `manufacture_year` that have flown more than 10,000 flights.

In [220]:
delays_by_manufacture_year_df = flight_df.groupby('manufacture_year')['is_delayed'].agg(['mean', 'count']).reset_index()

In [221]:
fig = px.scatter(delays_by_manufacture_year_df[delays_by_manufacture_year_df['count'] > 10000], 
                    x = 'manufacture_year',
                    y = 'mean',
                    title= 'Manufacture Year vs. Delay'
                   )

fig.update_xaxes(title_text='Manufacture Year')

fig.update_yaxes(title_text='Delayed Flights [%]')

fig.show()

Adding a `manufacture_year_agg` column to the `flight_df` frame.

In [222]:
flight_df['manufacture_year_agg'] = pd.cut(flight_df['manufacture_year'], bins = range(int(min(flight_df['manufacture_year'])), int(max(flight_df['manufacture_year'])) + 3, 3), right = True)

In [223]:
delays_by_manufacture_year_df.sample()

,manufacture_year,mean,count
1,1988.0,0.181388,634


Creation of the variable `flight_delays_by_manufacture_year_agg_df`.

In [ ]:
flight_delays_by_manufacture_year_agg_df = flight_df.groupby('manufacture_year_agg')['is_delayed'].agg(['mean', 'count']).reset_index()

In [229]:
flight_delays_by_manufacture_year_agg_df['manufacture_year_agg'] = flight_delays_by_manufacture_year_agg_df['manufacture_year_agg'].astype(str)

Chart based on data in `flight_delays_by_manufacture_year_agg_df`.

In [230]:
fig = px.scatter(flight_delays_by_manufacture_year_agg_df, 
                    x = 'manufacture_year_agg',
                    y = 'mean',
                    title= 'Manufacture Year vs. Delay'
                   )

fig.update_xaxes(title_text='Manufacture Year')

fig.update_yaxes(title_text='Delayed Flights [%]')

fig.show()

TOP 5 production years with the most flights

In [237]:
top_manufactured_df = delays_by_manufacture_year_df.sort_values('count', ascending=False).head()

In [238]:
top_manufactured_df.head()

,manufacture_year,mean,count
14,2001.0,0.154113,100251
29,2016.0,0.186717,66191
30,2017.0,0.187289,62353
27,2014.0,0.178658,61128
28,2015.0,0.189214,58426


Saving the frame to .cvs

In [239]:
flight_df.to_csv('flight_df_02.csv', index=False)